In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Linear_Regression").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/09 20:03:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

24/03/09 20:04:21 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
training.show(truncate=False)

+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label              |features                                                                                                                                                                                                                                |
+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|-9.490009878824548 |(10,[0,1,2,3,4,5,6,7,8,9],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658

In [8]:
lr = LinearRegression(
    featuresCol="features", labelCol="label", predictionCol="prediction"
)

In [9]:
lrModel = lr.fit(training)

24/03/09 20:07:38 WARN Instrumentation: [c4cd937e] regParam is zero, which might cause numerical instability and overfitting.
24/03/09 20:07:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/03/09 20:07:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/03/09 20:07:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [10]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [11]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

In [13]:
training_summary.r2

0.027839179518600154

In [14]:
training_summary.rootMeanSquaredError

10.16309157133015

train-test split

In [28]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

24/03/09 20:20:03 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [29]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [30]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
+----------

In [31]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
|-15.437384793431217|(10,[0,1,2,3,4,5,...|
|-15.359544879832677|(10,[0,1,2,3,4,5,...|
|-15.310980589416289|(10,[0,1,2,3,4,5,...|
|-13.976130931152703|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
|-12.500773785355054|(10,[0,1,2,3,4,5,...|
| -11.87816749996684|(10,[0,1,2,3,4,5,...|
| -11.43180236554046|(10,[0,1,2,3,4,5,...|
|-11.328415936777782|(10,[0,1,2,3,4,5,...|
|-10.945919657782932|(10,[0,1,2,3,4,5,...|
|-10.489157123372898|(10,[0,1,2,3,4,5,...|
| -10.35591860037468|(10,[0,1,2,3,4,5,...|
+----------

In [32]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 341|
|   mean|-0.18636461963935932|
| stddev|  10.513866503373183|
|    min| -28.571478869743427|
|    max|   27.78383192005107|
+-------+--------------------+



In [33]:
test_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|               160|
|   mean|1.2015727201044533|
| stddev| 9.852850462517791|
|    min|-23.51088409032297|
|    max|22.923352376063196|
+-------+------------------+



In [34]:
correct_model = lr.fit(train_data)

24/03/09 20:20:53 WARN Instrumentation: [b208c4be] regParam is zero, which might cause numerical instability and overfitting.


In [35]:
test_result = correct_model.evaluate(test_data)

In [36]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-22.259302835730377|
| -19.12240890985951|
| -20.32235357932748|
|-19.232629003338435|
| -18.42479312934593|
|-16.141614587799197|
|-15.636962729183857|
|-18.029141698589413|
|-15.262023555950886|
|-16.996222900312365|
| -12.92802135141816|
|-13.849997208107911|
|-16.092354599760174|
|-13.976585746182886|
|-14.076214062116165|
|-13.220071768214217|
|-12.652513447965337|
|-10.622494812533045|
|-10.042127498703747|
| -11.24497677145192|
+-------------------+
only showing top 20 rows



In [37]:
test_result.rootMeanSquaredError

9.784730735658814

In [38]:
unlabeled_data = test_data.select("features")

In [39]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [40]:
predictions = correct_model.transform(unlabeled_data)

In [41]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.2515812545925933|
|(10,[0,1,2,3,4,5,...|  -3.715051507059832|
|(10,[0,1,2,3,4,5,...|  0.4377928050540569|
|(10,[0,1,2,3,4,5,...|  -0.640362034729971|
|(10,[0,1,2,3,4,5,...| -0.9775429008686238|
|(10,[0,1,2,3,4,5,...| -1.3525857690841485|
|(10,[0,1,2,3,4,5,...| -1.3895295350256902|
|(10,[0,1,2,3,4,5,...|  1.3369346772783062|
|(10,[0,1,2,3,4,5,...|  -0.175361237480331|
|(10,[0,1,2,3,4,5,...|  1.6366780204796885|
|(10,[0,1,2,3,4,5,...| -2.3829592379981293|
|(10,[0,1,2,3,4,5,...|-0.12613372304479065|
|(10,[0,1,2,3,4,5,...|   2.319913038057304|
|(10,[0,1,2,3,4,5,...|  1.4758119608278315|
|(10,[0,1,2,3,4,5,...|   2.198046562149324|
|(10,[0,1,2,3,4,5,...|  1.7882694026737564|
|(10,[0,1,2,3,4,5,...|  1.3240975111875544|
|(10,[0,1,2,3,4,5,...| -0.3234248452498865|
|(10,[0,1,2,3,4,5,...| -0.4470296246691516|
|(10,[0,1,2,3,4,5,...|  0.889058